In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

# Load your dataset
df = pd.read_csv('it_problems.csv')

# Tokenize the problem descriptions
tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['problem_description'])
sequences = tokenizer.texts_to_sequences(df['problem_description'])

# Pad the sequences to ensure uniform input length
x_data = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=200)

# Convert the solutions to categorical data
y_data = pd.get_dummies(df['solution']).values

# Build the model
model = keras.Sequential([
    layers.Embedding(input_dim=10000, output_dim=64, input_length=200),
    layers.LSTM(64),
    layers.Dense(64, activation='relu'),
    layers.Dense(y_data.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
history = model.fit(x_data, y_data, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(x_data, y_data)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

# Make predictions
new_problems = ["Example problem description"]
new_sequences = tokenizer.texts_to_sequences(new_problems)
new_padded = keras.preprocessing.sequence.pad_sequences(new_sequences, maxlen=200)

predictions = model.predict(new_padded)
predicted_solution = np.argmax(predictions, axis=1)

print(predicted_solution)

# Save the model
model.save('it_problem_solver_model.h5')


In [1]:
import pandas as pd

# Define the data for the dataset
data = {
    'problem_description': [
        "How to fix a '404 Not Found' error?",
        "How to recover a lost password on Windows?",
        "How to install Python on Linux?",
        "How to set up a virtual environment in Python?",
        "How to resolve a 'Connection refused' error in a web server?"
    ],
    'correct_answers': [
        "Check if the URL is correct or if the page has been moved.|Ensure the server is running and the path is correct.|Clear the browser cache and refresh.",
        "Use the password reset feature on the login screen.|Follow the steps provided in the recovery email.|Use the Windows security questions if set up.",
        "Use the package manager to install Python, e.g., `sudo apt-get install python3`.|Download and install from the official Python website.|Use `conda install python` if using Anaconda.",
        "Use `venv` or `virtualenv` to create an isolated environment.|Run `python -m venv myenv` to set up the environment.|Activate the environment with `source myenv/bin/activate`.",
        "Check if the web server is running and if the firewall settings are correct.|Verify that the server is listening on the correct port.|Check server logs for errors."
    ],
    'important_words': [
        "404,Not Found,error",
        "recover,lost,password,Windows",
        "install,Python,Linux",
        "set up,virtual environment,Python",
        "resolve,Connection refused,web server"
    ]
}

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Explode the 'correct_answers' column
df_correct_answers = df.set_index('problem_description')['correct_answers'].str.split('|', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='correct_answer')

# Explode the 'important_words' column
df_important_words = df.set_index('problem_description')['important_words'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='important_words')

# Merge the exploded DataFrames
df_exploded = df_correct_answers.merge(df_important_words, on='problem_description')

# Save the DataFrame to a CSV file
df_exploded.to_csv('it_problems_with_exploded_answers_and_important_words.csv', index=False)
print("CSV file 'it_problems_with_exploded_answers_and_important_words.csv' has been created.")


CSV file 'it_problems_with_exploded_answers_and_important_words.csv' has been created.


In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
from tqdm import tqdm

# Load the dataset
df = pd.read_csv('it_problems_with_exploded_answers_and_important_words.csv')

# Combine all texts (questions, answers, and important words) for tokenization
all_texts = df['problem_description'].tolist() + df['correct_answer'].tolist() + df['important_words'].str.split(',').explode().tolist()

# Tokenize and pad the texts
max_length = 100
vocab_size = 10000
embedding_dim = 64

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(all_texts)
sequences = tokenizer.texts_to_sequences(all_texts)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Split the sequences back into questions, answers, and important words
question_sequences = padded_sequences[:len(df)]
answer_sequences = padded_sequences[len(df):len(df) * 2]
important_words_sequences = padded_sequences[len(df) * 2:]

# Define the Bahdanau Attention layer
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)
    
    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

# Define the model with attention and dynamic embedding adjustment
class AttentionModel(Model):
    def __init__(self, vocab_size, embedding_dim, units):
        super(AttentionModel, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.lstm = Bidirectional(LSTM(units, return_sequences=True, return_state=True))
        self.attention = BahdanauAttention(units)
        self.dense = Dense(units, activation='relu')
    
    def call(self, x, important_weights=None):
        x = self.embedding(x)
        
        if important_weights is not None:
            x = x * important_weights

        lstm_out, forward_h, forward_c, backward_h, backward_c = self.lstm(x)
        state_h = tf.concat([forward_h, backward_h], axis=1)
        context_vector, attention_weights = self.attention(state_h, lstm_out)
        output = self.dense(context_vector)
        return output

units = 64
model = AttentionModel(vocab_size, embedding_dim, units)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Prepare the important words indices for each question-answer pair
important_words_indices = []
for words in df['important_words']:
    indices = [tokenizer.word_index.get(word) for word in words.split(',') if tokenizer.word_index.get(word) is not None]
    important_words_indices.append(indices)

# Train the model using the real target embeddings and important words
answer_embeddings = model(answer_sequences).numpy()

epochs = 10

for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    with tqdm(total=len(question_sequences), desc='Training') as pbar:
        for i in range(len(question_sequences)):
            important_indices = np.array(important_words_indices[i:i+1], dtype=object)
            with tf.GradientTape() as tape:
                outputs = model(question_sequences[i:i+1], important_weights=None)
                loss = tf.keras.losses.mean_squared_error(answer_embeddings[i:i+1], outputs)
            gradients = tape.gradient(loss, model.trainable_variables)
            model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            pbar.update(1)

# Save the model in the SavedModel format
model.save('saved_attention_model', save_format='tf')
print("Model saved as 'saved_attention_model'")

Epoch 1/10


100%|██████████| 2/2 [00:02<00:00,  1.14s/it]


Epoch 2/10


100%|██████████| 2/2 [00:01<00:00,  1.00it/s]


Epoch 3/10


...

Epoch 9/10


100%|██████████| 2/2 [00:01<00:00,  1.02it/s]


Epoch 10/10


100%|██████████| 2/2 [00:02<00:00,  1.04s/it]


TypeError: in user code:

    File "d:\Software\environments\deep_learning\environments\deep_learning\lib\site-packages\keras\saving\saving_utils.py", line 147, in _wrapped_model  *
        outputs = model(*args, **kwargs)
    File "d:\Software\environments\deep_learning\environments\deep_learning\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\ASUS\AppData\Local\Temp\__autograph_generated_filev0pfdyx4.py", line 26, in tf__call
        ag__.if_stmt(ag__.ld(important_weights) is not None, if_body, else_body, get_state, set_state, ('x',), 1)
    File "C:\Users\ASUS\AppData\Local\Temp\__autograph_generated_filev0pfdyx4.py", line 21, in if_body
        x = ag__.ld(x) * ag__.ld(important_weights)

    TypeError: Exception encountered when calling layer "attention_model_1" "                 f"(type AttentionModel).
    
    in user code:
    
        File "C:\Users\ASUS\AppData\Local\Tem

In [33]:
def get_embeddings(sequences, important_words_indices):
    # Tokenize and pad the sequences
    sequences = tokenizer.texts_to_sequences(sequences)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

    # Ensure the shapes are compatible for tensor_scatter_nd_update
    batch_size = padded_sequences.shape[0]
    seq_length = padded_sequences.shape[1]
    hidden_dim = 64  # Example hidden dimension size; adjust as necessary

    # Create an initial tensor of ones with the desired shape
    important_weights = tf.ones((batch_size, seq_length, hidden_dim))

    # Update the tensor based on important words indices
    for i, indices in enumerate(important_words_indices):
        if len(indices) > 0:
            indices = np.array(indices)
            indices = indices[indices < seq_length]
            updates = tf.ones((len(indices), hidden_dim)) * 2.0
            updates = tf.reshape(updates, (len(indices), hidden_dim))
            important_weights = tf.tensor_scatter_nd_update(important_weights, tf.constant([[i, idx] for idx in indices], dtype=tf.int32), updates)

    # Perform the forward pass
    embeddings = model(padded_sequences, important_weights=important_weights).numpy()
    return embeddings

def find_most_similar_answer(answer, correct_answers, important_words):
    answers_to_compare = correct_answers + [answer]
    important_words_indices = [
        [tokenizer.word_index[word] for word in answer.split() if word in important_words and word in tokenizer.word_index] 
        for answer in answers_to_compare
    ]
    embeddings = get_embeddings(answers_to_compare, important_words_indices)
    similarities = cosine_similarity([embeddings[-1]], embeddings[:-1])[0]
    most_similar_index = np.argmax(similarities)
    return most_similar_index, similarities[most_similar_index]

def ask_question(question, correct_answers, important_words):
    print(f"Question: {question}")
    answer1 = input("Player 1, your answer: ")
    answer2 = input("Player 2, your answer: ")
    index1, similarity1 = find_most_similar_answer(answer1, correct_answers, important_words)
    index2, similarity2 = find_most_similar_answer(answer2, correct_answers, important_words)
    return similarity1, similarity2, index1, index2

In [ ]:
def display_menu():
    print("Welcome to the IT Problem-Solving Game!")
    print("1. Start Game")
    print("2. Exit")
    choice = input("Enter your choice: ")
    return choice

def main():
    while True:
        choice = display_menu()
        if choice == '1':
            question_row = df.sample(n=1).iloc[0]
            question = question_row['problem_description']
            correct_answers = df[df['problem_description'] == question]['correct_answer'].tolist()
            important_words = df[df['problem_description'] == question]['important_words'].tolist()
            similarity1, similarity2, index1, index2 = ask_question(question, correct_answers, important_words)
            if similarity1 > similarity2:
                print(f"Player 1 wins![{similarity1}|{similarity2}]")
            elif similarity2 > similarity1:
                print(f"Player 2 wins![{similarity1}|{similarity2}]")
            else:
                print(f"TIE! [{similarity1}|{similarity2}]")
        elif choice == '2':
            print("Thanks for playing!")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()